In [1]:
import sys
sys.path.insert(0, r'C:\Users\woyao\Desktop\Cambridge\ym432')  # adjust to your project root
%load_ext line_profiler

In [2]:
import linecache, sys
for mod in list(sys.modules):
    if mod.startswith("pauli_propagation"):
        del sys.modules[mod]
linecache.clearcache()

In [3]:
from pauli_propagation.propagator import PauliPropagator
from pauli_propagation.utils      import weight_of_key, encode_pauli, random_su4
from pauli_propagation.gates      import QuantumGate
from pauli_propagation.pauli_term import PauliTerm

from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info    import Pauli

import random


In [4]:
def _staircase_edges(nx, ny):
    next_inds, temp_inds, edges = [1], [], []
    while next_inds:
        for i in next_inds:
            if i % nx != 0:
                edges.append((i, i+1))
                temp_inds.append(i+1)
            if ((i-1)//nx + 1) < ny:
                edges.append((i, i+nx))
                temp_inds.append(i+nx)
        next_inds, temp_inds = temp_inds, []
    seen, uniq = set(), []
    for e in edges:
        if e not in seen:
            seen.add(e)
            uniq.append(e)
    return uniq

def staircasetopology2d_qc(nx, ny):
    nqubits = nx*ny
    qc = QuantumCircuit(nqubits)
    for k, (q1, q2) in enumerate(_staircase_edges(nx, ny)):
        mat  = random_su4()
        gate = UnitaryGate(mat, label=f"SU4_{k}")
        gate._name = "su4"
        qc.append(gate, [q1-1, q2-1])
    return qc


In [5]:
nx, ny = 4, 4
qc_2d = staircasetopology2d_qc(nx, ny)

n = qc_2d.num_qubits
pauli_label = "X" + "I"*(n-1)
key = encode_pauli(Pauli(pauli_label))
init_term = PauliTerm(1.0, key, n)

prop_2d = PauliPropagator(qc_2d)

In [ ]:
# %%
# Cell 6: line‐by‐line profiling of propagate()
%lprun -f PauliPropagator.propagate \
       -f weight_of_key \
       -f QuantumGate.get \
       _ = prop_2d.propagate(init_term, max_weight=3)

Propagating:   0%|          | 0/24 [00:00<?, ?it/s]